# Module 2 Part 2: The Machine Learning Process, a Worked Example

This module consists of 2 parts:

- **Part 1** - The Machine Learning Process, a Worked Example (Data Exploration)

- **Part 2** - The Machine Learning Process, a Worked Example (Data Modelling)

Each part is provided in a separate notebook file. It is recommended that you follow the order of the notebooks.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<br>
<div class="toc">
<ul class="toc-item">
<li><span><a href="#Module-2-Part-2:-The-Machine-Learning-Process,-a-Worked-Example" data-toc-modified-id="Module-2-Part-2:-The-Machine-Learning-Process,-a-Worked-Example">Module 2 Part 2: The Machine Learning Process, a Worked Example</a></span>
</li>
<li><span><a href="#Table-of-Contents" data-toc-modified-id="Table-of-Contents">Table of Contents</a></span>
</li>
<li><span><a href="#Data-Sources" data-toc-modified-id="Data-Sources">Data Sources</a></span>
</li>
<li><span><a href="#Pre-processing-Pipelines" data-toc-modified-id="Pre-processing-Pipelines">Pre-processing Pipelines</a></span>
<ul class="toc-item">
<li><span><a href="#Imputers" data-toc-modified-id="Imputers">Imputers</a></span>
</li>
<li><span><a href="#Categorical-encoders" data-toc-modified-id="Categorical-encoders">Categorical encoders</a></span>
</li>
<li><span><a href="#Feature-scalers" data-toc-modified-id="Feature-scalers">Feature scalers</a></span>
</li>
<li><span><a href="#Custom-transformers" data-toc-modified-id="Custom-transformers">Custom transformers</a></span>
<ul class="toc-item">
<li><span><a href="#Variable-buckets" data-toc-modified-id="Variable-buckets">Variable buckets</a></span>
</li>
</ul>
</li>
<li><span><a href="#Bringing-the-pipeline-together" data-toc-modified-id="Bringing-the-pipeline-together">Bringing the pipeline together</a></span>
</li>
</ul>
</li>
<li><span><a href="#Train-test-Split" data-toc-modified-id="Train-test-Split">Train-test Split</a></span>
<ul class="toc-item">
<li><span><a href="#Stratified-Sampling" data-toc-modified-id="Stratified-Sampling">Stratified Sampling</a></span>
</li>
</ul>
</li>
<li><span><a href="#Basic-Regressions" data-toc-modified-id="Basic-Regressions">Basic Regressions</a></span>
</li>
<li><span><a href="#Advanced-Regressions" data-toc-modified-id="Advanced-Regressions">Advanced Regressions</a></span>
<ul class="toc-item">
<li><span><a href="#ElasticNet" data-toc-modified-id="ElasticNet">ElasticNet</a></span>
</li>
<li><span><a href="#SGDRegressor" data-toc-modified-id="SGDRegressor">SGDRegressor</a></span>
</li>
<li><span><a href="#DecisionTreeRegressor" data-toc-modified-id="DecisionTreeRegressor">DecisionTreeRegressor</a></span>
</li>
<li><span><a href="#EXERCISE-4:-Building-a-regression-model" data-toc-modified-id="EXERCISE-4:-Building-a-regression-model">EXERCISE 4: Building a regression model</a></span>
</li>
</ul>
</li>
<li><span><a href="#Cross-validation" data-toc-modified-id="Cross-validation">Cross-validation</a></span>
</li>
<li><span><a href="#Hyperparameter-Search" data-toc-modified-id="Hyperparameter-Search">Hyperparameter Search</a></span>
</li>
<li><span><a href="#Conclusion:-Launch,-Monitor,-Maintain" data-toc-modified-id="Conclusion:-Launch,-Monitor,-Maintain">Conclusion: Launch, Monitor, Maintain</a></span>
<ul class="toc-item">
<li><span><a href="#Launch" data-toc-modified-id="Launch">Launch</a></span>
</li>
<li><span><a href="#Monitor" data-toc-modified-id="Monitor">Monitor</a></span>
</li>
<li><span><a href="#Maintain" data-toc-modified-id="Maintain">Maintain</a></span>
</li>
<li><span><a href="#Taking-this-further" data-toc-modified-id="Taking-this-further">Taking this further</a></span>
</li>
</ul>
</li>
<li><span><a href="#Exercise-Solutions" data-toc-modified-id="Exercise-Solutions">Exercise Solutions</a></span>
<ul class="toc-item">
<li><span><a href="#EXERCISE-4" data-toc-modified-id="EXERCISE-4">EXERCISE 4</a></span>
</li>
</ul>
</li>
<li><span><a href="#References" data-toc-modified-id="References">References</a></span>
</li>
</ul>
</div>

# Data Sources

You can find the original dataset [here](http://insideairbnb.com/get-the-data.html).

In [36]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

def rmse(a, b):
    return np.sqrt(np.mean((a-b)**2))
pd.options.display.max_columns = None
# to make this notebook's output identical at every run
np.random.seed(0)

# Pre-processing Pipelines

In Part 1 of the module, we focused on exploring our data. We identified features that require pre-processing before they can be used in a machine learning model. We covered imputation, log transformations, and category binning. In addition to these changes, we will need to transform the data to satisfy the mathematical assumptions of our models. These changes include scaling our numerical data and encoding our categorical data.

The `scikit-learn` package has a [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) that can be used to handle various pre-processing steps. In addition to the pipeline, `pandas` formatted data can be transformed using the [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html). We will use these tools combined with multiple pre-processing techniques to transform the raw data into useable features for machine learning models.

To begin, we'll reload the dataset from the source file and separate the target variable.

In [37]:
df = pd.read_csv('../data/toronto_listings_119.csv', index_col='id')
# a bit of extra pre-processing
# others = [x for x in df['property_type'].value_counts().index if df['property_type'].value_counts()[x] <= 10]
# df['property_type'] = df['property_type'].apply(lambda x: 'Other' if x in others else x)

Let's review the dataset. It has some missing values, lots of categorical variables, and will require some data transformations.

In [38]:
df.sample(5)

,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,guests_included,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
30813194,f,2.0,t,f,43.760928,-79.411002,t,Apartment,Shared room,1,1.0,1.0,1.0,Real Bed,"{Wifi,""Air conditioning"",Elevator,Heating,Wash...",48.0,2,365,2,100.0,9.0,9.0,9.0,9.0,9.0,8.0,t,flexible,2.00
2823607,f,1.0,t,t,43.657402,-79.455290,t,Apartment,Entire home/apt,3,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Free par...",100.0,1,0,50,92.0,10.0,9.0,10.0,10.0,9.0,9.0,f,flexible,0.90
12835392,f,1.0,t,t,43.764857,-79.412793,t,Apartment,Entire home/apt,1,1.0,1.0,1.0,Real Bed,"{TV,Internet,Kitchen,""Free parking on premises...",145.0,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,flexible,0.03
28183298,f,2.0,t,t,43.685043,-79.399169,t,Apartment,Entire home/apt,1,1.0,0.0,1.0,Pull-out Sofa,"{Internet,Wifi,""Wheelchair accessible"",Kitchen...",80.0,1,358,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,strict_14_with_grace_period,NaN
19680294,f,1.0,t,f,43.617793,-79.487189,f,Apartment,Private room,3,1.0,1.0,1.0,Airbed,"{Wifi,Kitchen,""Pets live on this property"",Was...",50.0,1,0,8,90.0,9.0,9.0,10.0,9.0,9.0,9.0,t,flexible,0.48


Machine learning models generally work best with numerical data only. Therefore, we will need to clean up the data.

The first step of setting up our pre-processing pipeline will be identifying the various [`Transformer()`](https://scikit-learn.org/stable/data_transforms.html) classes that we will use. Transformers are Python classes that take `pandas` DataFrames / arrays of values as input, and output transformed arrays that may even have a different shape. 

The `sklearn` module provides many useful transformers, as well as the ability to create custom transformers. The transformer classes are of the form `Transformer(arguments)` and we will use the `fit(data)`, `transform(data)`, and `fit_transform()` methods to process our data.

There are a few important requirements for a good machine learning dataset:

1. Features should not have any missing values.<br><br>

2. All features must be numerical.<br><br>

3. Features should have the same or similar scales (in particular, range and expected value).

Many machine learning models assume these requirements are met and will not perform well with data that does not adhere to them.

Next, we will cover the main types of transformers necessary for our pipeline. Afterwards, we will show how to link them together to build a pipeline and apply them to our data features.

## Imputers

The first assumption is that our data has no missing values. During exploration, we discovered this is not the case. There are a number of ways that we can work with missing data. In Part 1 of the module, we simply dropped records where values were missing. This strategy can work in certain circumstances, but we may find that we lose valuable data.

Alternatively, we can fill in the missing data according to an appropriate rule. A variety of imputation rules exist. One option is to assign a placeholder value indicating missing data. For example, an "other" category. We could also fill in these values according to a measure of central tendency (mean, median, or mode). Or, in special cases, such as time series, we could interpolate values based on the values around them.

For **imputation**, we will use the [`SimpleImputer()`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) transformer. This class offers a variety of options we can use for different types of features. For our categorical values, we can assign missing values to an "other" category &mdash; e.g. `SimpleImputer(strategy='constant', fill_value='Other')`. For our numerical values, we can use the "median" method (`SimpleImputer(strategy='median')`).

#### List of all columns which has missing values before the transformation

In [39]:
df.isna().sum()[df.isna().sum() > 0]

host_is_superhost                 3
host_listings_count               3
host_has_profile_pic              3
host_identity_verified            3
bathrooms                        14
bedrooms                          7
beds                             21
review_scores_rating           4034
review_scores_accuracy         4044
review_scores_cleanliness      4043
review_scores_checkin          4048
review_scores_communication    4042
review_scores_location         4051
review_scores_value            4050
reviews_per_month              3768
dtype: int64

#### Check property type feature

In [40]:
print(df[['property_type']].values[:, 0])
print(df[['accommodates']].values[:, 0])

['House' 'Apartment' 'House' ... 'House' 'Apartment' 'Condominium']
[10  2  1 ...  2  4  2]


#### Transfrom missing values for property_type with 'Other' and accommodates with median

In [41]:
tfmr = SimpleImputer(strategy='constant', fill_value='Other')
# we place double brackets as a trick - sklearn expects arrays of the shape (n, 1). 
# the pandas Series.values method gives the shape (n, ), so we pass a DataFrame instead
# by including a list of one column header.
print(tfmr.fit_transform(df[['property_type']].values)[:, 0])

print(df[['accommodates']].values[:, 0])
tfmr = SimpleImputer(strategy='median')
print(tfmr.fit_transform(df[['accommodates']].values)[:, 0])

['House' 'Apartment' 'House' ... 'House' 'Apartment' 'Condominium']
[10  2  1 ...  2  4  2]
[10.  2.  1. ...  2.  4.  2.]


#### Check how many records are transformed

In [42]:
print(str(df[df['property_type']=='Other'].shape[0]))
print(str(df[df['accommodates'] % 1 != 0].shape[0]))

60
0


#### List of all columns which has missing values after the transformation

In [43]:
df.isna().sum()[df.isna().sum() > 0]

host_is_superhost                 3
host_listings_count               3
host_has_profile_pic              3
host_identity_verified            3
bathrooms                        14
bedrooms                          7
beds                             21
review_scores_rating           4034
review_scores_accuracy         4044
review_scores_cleanliness      4043
review_scores_checkin          4048
review_scores_communication    4042
review_scores_location         4051
review_scores_value            4050
reviews_per_month              3768
dtype: int64

## Categorical encoders

Since all of our data has to be numerical, we need a mechanism to transform our categorical data. For our categorical features, we can use **one-hot encoding** to represent the different categories numerically. With this approach, each unique value of a feature is given its own column. Thus, the data will be encoded with a "1" in the column which matches the value and a "0" in every other column.

The use of multiple columns via one-hot encoding ensures that our categorical values are not treated as ordinal data. For example, if instead we used the values "1" and "2" to represent "bed" and "floor" in a single column, a model might consider these values as more similar than the values "1" and "4" representing "bed" and "cot." We don't want to introduce additional associations between values through the choice of encoding scheme.

We will use the [`OneHotEncoder(sparse=False)`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) transformer. We specify `sparse=False` to avoid outputting a sparse `scipy` matrix. A spare matrix can be extremely useful for handling very large datasets with mostly zero entries. However, for ease of understanding, we will stick to `numpy` arrays for this module.

In [44]:
print(df[['property_type']].values)
tfmr = OneHotEncoder(sparse_output=False)
print(tfmr.fit_transform(df[['property_type']].values))

[['House']
 ['Apartment']
 ['House']
 ...
 ['House']
 ['Apartment']
 ['Condominium']]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Feature scalers
Using different features that have hugely different scales can be problematic for some machine learning models. Therefore, it is good practice to implement a **scaling function** on numerical values. As an example, our `latitude` feature has a very small range and very high mean relative to a feature such as `accommodates`.

There are two main scaling methods:

1. **Normalization** or *min/max scaling* shifts and rescales data points between the values 0 and 1. This method is more likely to be affected by outliers.<br><br>

2. **Standardization** works best with normally distributed data by subtracting the mean value from each data point and dividing by the standard deviation.

There is no guaranteed rule for choosing between these methods, but data exploration can help to identify features that may be problematic for either method.

We will use the [`MinMaxScaler()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) on all of our numerical (discrete and continuous) features. Our exploration did not reveal many outliers and it is best to use a consistent scaling method. If you would like to experiment, you can replace the `MinMaxScaler()` with the [`StandardScaler()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) and examine the impact on the models.

In [45]:
print(df[['accommodates']].values[:, 0])
tfmr = MinMaxScaler()
print(tfmr.fit_transform(df[['accommodates']].values)[:, 0])

print(df[['latitude']].values[:, 0])
tfmr = StandardScaler()
print(tfmr.fit_transform(df[['latitude']].values)[:, 0])

# NOTE: If you get a DataConversionWarning you can safely ignore it.

[10  2  1 ...  2  4  2]
[0.6        0.06666667 0.         ... 0.06666667 0.2        0.06666667]
[43.64616766 43.64105127 43.66724069 ... 43.67788972 43.66907246
 43.63804609]
[-0.69774523 -0.80533284 -0.25462118 ... -0.03069313 -0.21610266
 -0.8685259 ]


## Custom transformers
There are a great variety of transformers available in the `sklearn` package. However, if you can't find a transformer that suit your needs, it is straightforward to create your own! Any function can be easily wrapped in a tranformer class using the [`FunctionTransformer()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html).

Here is a simple example:

In [46]:
def plusone(x):
    return x + 1

# print the original values
print(df[['accommodates']].values[:, 0])

# apply the transformation
tfmr = FunctionTransformer(func=plusone, validate=False)

# print the values plus one
print(tfmr.fit_transform(df[['accommodates']].values)[:, 0])

[10  2  1 ...  2  4  2]
[11  3  2 ...  3  5  3]


### Variable buckets

We should sort our features according to the different transformations each feature will need. In the code below, there are four different buckets.

1. The categorical variables will need to be imputed and encoded.<br><br>

2. The discrete variables will be imputed and scaled.<br><br>

3. The continuous variables will also be imputed and scaled.<br><br>

4. The dependent variable `price` will be imputed.

**NOTE:** The dependent variable typically does not need to be scaled. And, for our purposes, avoiding scaling will make it easier to understand our models' performance.

In [47]:
# Copied from Part 1, our data types
categorical_vars = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified',
            'is_location_exact', 'property_type', 'room_type', 'bed_type', 
            'instant_bookable', 'cancellation_policy']

discrete_vars = ['host_listings_count', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
                 'guests_included', 'availability_365']

continuous_vars = ['latitude', 'longitude']

dep_var = ['price']

The workflow for `ColumnTransformer` pipelines generally includes these steps:

1. **Transformers**: In this case, the ones we have already discussed.<br><br>

2. **Pipelines**: These link together sequential transformers, making a chain of transformers that will each take in an input array or matrix and pass on the transformed output. The [`Pipeline()`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) function takes as input a list of `(name, transformer)` tuples.<br><br>

3. **ColumnTransformer**: This function will apply any number of pipelines to designated columns of a 2-dimensional data structure, making it suitable for working with `pandas` DataFrames. By indicating which pipelines apply to which columns, we can build a concise workflow that works with a variety of features that require different transformations. The [`ColumnTransformer()`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) pipeline takes as input a list of `(name, pipeline, column_headers)` tuples.

## Bringing the pipeline together

Here are the transformation pipelines for the categorical and independent variables.

In [48]:
# Categorical - impute, one hot encode
cat_si_step = ('si', SimpleImputer(strategy='constant', fill_value='Other'))
cat_ohe_step = ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
cat_steps = [cat_si_step, cat_ohe_step]
cat_pipe = Pipeline(cat_steps)
cat_transformers = [('cat', cat_pipe, categorical_vars)]

# Numerical - impute, scale
num_si_step = ('si', SimpleImputer(strategy='median'))
num_scl_step = ('scl', MinMaxScaler())
num_steps = [num_si_step, num_scl_step]
num_pipe = Pipeline(num_steps)
num_transformers = [('num', num_pipe, discrete_vars + continuous_vars)]

Now, we can use our pipelines to transform our data.

In [49]:
ct = ColumnTransformer(transformers=cat_transformers + num_transformers)
ct.fit(df[categorical_vars + discrete_vars + continuous_vars])
X = ct.transform(df[categorical_vars + discrete_vars + continuous_vars])
# We know from our exploration that the dependent variable 'price' does not have any missing values. 
# It is also generally not necessary to apply transformations to normalize or scale
# the dependent variable.
y = df[['price']].values

# Train-test Split

When creating machine learning models, it is good practice to split your data into subsets: training data and testing data (we can also include a validation subset). This should be done randomly to avoid introducing bias. We fit the model on the training data and test its prediction strength on the testing data using our performance metric. This can help us identify two potential problems: overfitting or underfitting the model.

1. **Overfitting** is when a model has been too finely tuned towards the training dataset. This issue can often arise in cases where there are relatively few datapoints compared to the number of features. The model may be finding many relationships that are not actually generalizable beyond the training dataset, giving it poor performance on a test dataset that it has not seen before.<br><br>

2. **Underfitting** occurs when a model does not fit the training data well and is unable to find trends. Underfitting is more likely to occur in the case of a dataset that is too simple (few relevant independent variables) or when there is a mismatch between the data and the model chosen (e.g. a linear model for a non-linear relationship).

Sklearn has the useful [`train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function for this purpose. We will split our independent $X$ values and dependent $y$ values each into a train set and a test set. A 20% split is a typical parameter to use.

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(15717, 68) (3930, 68) (15717, 1) (3930, 1)


## Stratified Sampling

Random sampling is an effective way to avoid **sampling bias**, which occurs when certain groups in a dataset are over- or under-sampled. However, in small datasets random sampling can miss key structure in the data that is important to our research question, especially if we are interested in the tail ends of our dependent variable (e.g. the highest price AirBnB offerings). We can aim to preserve the overall distribution of data in our train and test datasets by using **stratified sampling**. 

For stratified sampling we segment our dataset into equally spaced **strata** and maintain the proportion of data in each stratum for our subsets.

In Part 1 of the module, we saw that the `latitude` feature follows a somewhat bimodal distribution. Intuitively, we should also expect latitude to be an important feature &mdash; e.g. the farther south we are in Toronto, the closer to downtown. To ensure that this feature is sampled in a way that maintains its proportions, let's try stratified sampling.

Sklearn offers the [`StratifiedShuffleSplit()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html) class to make stratified sampling easy. We will use the `split()` method and provide our dataset and the chosen feature as arguments. The method will maintain even proportions across the chosen feature in the train and test datasets. We will first have to make a new column to create strata for our continuous latitude variable. In this case, we can simply multiply by 10 and take the ceiling (`[np.ceil()`](https://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.ceil.html) to get five equally spaced and discrete strata.

In [51]:
df['lat_cat'] = np.ceil(df['latitude']*10)
print(df['lat_cat'].sample(frac=.2, random_state=0).value_counts())

lat_cat
437.0    3017
438.0     838
439.0      57
436.0      17
Name: count, dtype: int64


Now, we can use the `split()` method. This will provide us with indices that we can use to select from our $X$ and $y$ data, even though we got the indices from our original `df`. Our output should show dimensions of four arrays. The first two will have the same number of columns (all of our independent variables) and the second two will have one column (our dependent variable). They will be split in a ratio of 4:1 (or 0.8:0.2).

In [52]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

# note that we pass the dataframe as arguments to the .split() method 
# rather than the numpy matrix X. This is done for easy comprehension
# as the numpy matrix no longer has labelled columns. You can compare the
# shapes of X and df to ensure we are getting the right rows.
for train_index, test_index in split.split(df, df['lat_cat'].fillna(df['lat_cat'].median())):
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
        
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(15717, 68) (3930, 68) (15717, 1) (3930, 1)


# Basic Regressions

We are now ready to build some models! At this stage in the course, we will not focus on the details of each model. Different types of regression models and their strengths and weaknesses will be covered in a later module. However, it is good practice to try a few different models for comparison. Below we will implement a simple linear regression using `sklearn`'s [`LinearRegression()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) model. The widely used ordinary least squares (OLS) regression is a tried and true method for picking up relationships in data and using these relationships to make predictions.

In [53]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train, np.ravel(y_train))

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


We can write a simple function to print out our results, including our accuracy metric ($\text{RMSE}$), and a sample of 5 predicted and actual values.

In [54]:
def display_results(model, X, y):
    print("RMSE:", rmse(model.predict(X), y))
    print("Predicted 1-5:", model.predict(X_test[0:5]))
    print("Actual 1-5:", y_test[0:5, 0])
    
display_results(reg, X, y)

RMSE: 245.2729280277787
Predicted 1-5: [ 50.27816008  75.82746176 143.86736309  55.75755496  64.58525762]
Actual 1-5: [ 45.  38.  76.  69. 100.]


# Advanced Regressions

Next, we will implement the more advanced ElasticNet regression model ([`ElasticNet()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet)), as well as the stochastic gradient descent regressor ([`SGDRegressor()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor)) and the random forest regressor ([`RandomForestRegressor()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)). Each of these methods takes a different approach towards regression. Check out the [sklearn generalized linear models](https://scikit-learn.org/stable/modules/linear_model.html) documentation for an overview of these models and many others. You can also try implementing one of the other models described on that page.

## ElasticNet

The ElasticNet model implements methods to help with assumptions made in OLS regression. You do not need to worry about the details for now.

In [55]:
from sklearn.linear_model import ElasticNet

enr = ElasticNet()
enr.fit(X_train, np.ravel(y_train))
display_results(enr, X, y)

RMSE: 229.0536008162524
Predicted 1-5: [ 97.60287359 107.13705813 161.25775042 111.17237008  95.65962275]
Actual 1-5: [ 45.  38.  76.  69. 100.]


The ElasticNet performs better than the OLS regression, with a lower $\text{RMSE}$.

## SGDRegressor

Next, we will try the SGDRegressor, an old optimization method that has more recently become a standard component  of artificial neural networks (along with [backpropogation](https://en.wikipedia.org/wiki/Backpropagation)). 

In [56]:
from sklearn.linear_model import SGDRegressor
clf = SGDRegressor(tol=1e-3)
clf.fit(X_train, np.ravel(y_train))
display_results(clf, X, y)

RMSE: 242.03274785525838
Predicted 1-5: [ 61.10641599  78.43761646 136.30303542  65.25874821  55.8051915 ]
Actual 1-5: [ 45.  38.  76.  69. 100.]


The SGDRegressor performs only slightly better than OLS regression based on its $\text{RMSE}$ value.

## DecisionTreeRegressor

Decision tree models create "branches" at different feature values &mdash; e.g. a private home will lead to a prediction of a higher price. Decision trees and related models are valuable for ease of interpretation &mdash; the `sklearn` class makes it easy to determine which features were most important in the training of the model.

In [57]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(X_train, np.ravel(y_train))
display_results(dtr, X, y)

RMSE: 307.7835377080111
Predicted 1-5: [ 49.  84. 102.  76.  31.]
Actual 1-5: [ 45.  38.  76.  69. 100.]


We see some notable underfitting in the DecisionTreeRegressor based on its higher $\text{RMSE}$ value.

## EXERCISE 4: Building a regression model

Check out the list of regression models available in [`sklearn`](https://scikit-learn.org/stable/modules/linear_model.html). Try implementing one not listed above, and run it multiple times with some different parameters.

If you are interested in understanding some of the improvements made in the ElasticNet model, try the [`Ridge`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) or [`Lasso`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) models.

If you would like to improve on the DecisionTreeRegressor you could try the [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html), an ensemble model that fits multiple decision tree regressors over subsets of the data. This is most effective when you have overfitting in your decision tree, but can still improve on $\text{RMSE}$ in this case.

In [70]:
# Your code here
# from sklearn import linear_model
# lm = linear_model.Ridge(alpha=1.0)
# lm.fit(X_train, np.ravel(y_train))
# display_results(lm, X, y)

# lm = linear_model.Lasso(alpha=1.0)
# lm.fit(X_train, np.ravel(y_train))
# display_results(lm, X, y)

from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators=100, random_state=0)
rfr.fit(X_train, np.ravel(y_train))
display_results(rfr, X, y)


RMSE: 275.0999001899067
Predicted 1-5: [ 65.22  49.03 133.1   75.38  44.83]
Actual 1-5: [ 45.  38.  76.  69. 100.]


# Cross-validation

Splitting our data into a train set and a test set is a good start to avoiding overfitting. However, for a more comprehensive assessment of model performance we should cross-validate our models.

**K-fold cross validation** is a method where we randomly split our data into $k$ subsets called **folds**. We then train our model $k$ times, each time picking one fold as our validation or test set and training on the remaining subsets. This method greatly reduces sampling bias and gives us an estimate not just of average model performance, but also the standard deviation of model performance. This extra information allows us to assess both the precision and accuracy of our model.

We can use `sklearn`'s `cross_val_score()` function for easy cross-validation. Sklearn provides a number of different options for performance metrics. We will select `'neg_mean_squared_error'` and keep in mind that we will have to take the negative and square root of the result to get $\text{RMSE}$. Remember that the lower the $\text{RMSE}$ is, the better our model is doing.

In [59]:
from sklearn.model_selection import cross_val_score
enr = ElasticNet()
scores = cross_val_score(enr, X, np.ravel(y), cv=5, scoring='neg_mean_squared_error')
np.sqrt(-scores)

array([129.36841654, 325.20647905, 251.16609114, 215.60607491,
       128.32872966])

We can write a simple function to print out our results.

In [78]:
def display_scores(scores):
    print("Scores:", np.sqrt(-scores))
    print("Mean:", np.sqrt(-scores).mean())
    print("Standard deviation:", np.sqrt(-scores).std())

display_scores(scores)

Scores: [117.27319613 316.60660781 239.29366015 203.81316718 116.78433622]
Mean: 198.75419349809357
Standard deviation: 76.04793694064317


In [77]:
clf = SGDRegressor(max_iter=1000, tol=1e-3)
#SGDRegressor(alpha=np.float64(1e-05), max_iter=10000)
scores = cross_val_score(clf, X, np.ravel(y), cv=5, scoring='neg_mean_squared_error')
display_scores(scores)

Scores: [117.27319613 316.60660781 239.29366015 203.81316718 116.78433622]
Mean: 198.75419349809357
Standard deviation: 76.04793694064317


In [62]:
rfr = DecisionTreeRegressor()
scores = cross_val_score(dtr, X, np.ravel(y), cv=5, scoring='neg_mean_squared_error')
display_scores(scores)

Scores: [265.33405034 343.58032829 387.38323588 229.38951092 272.5925889 ]
Mean: 299.65594286585684
Standard deviation: 57.398986867388814


In [71]:
# You can add cross-validation for your model from exercise 4 here
rfr = RandomForestRegressor(n_estimators=100, random_state=0)
scores = cross_val_score(rfr, X, np.ravel(y), cv=5, scoring='neg_mean_squared_error')
display_scores(scores)

Scores: [181.80579366 309.22408608 294.85977551 203.5847255  134.32473285]
Mean: 224.7598227179125
Standard deviation: 67.11207641046593


While our ElasticNet model still looks effective, we actually find that the SGD regressor has, on average, a lower $\text{RMSE}$. Cross-validation can give you much more confidence in your calculation of performance metrics. 

# Hyperparameter Search

Once we have chosen a model type, it is important to tune its **hyperparameters**. The hyperparameters are the parameters that are decided before beginning the training process. We have already seen some hyperparameters as arguments above (e.g. for SGDRegressor). We will discard the assumption that these default hyperparameters are the best possible options, and we will instead test different parameters to determine which works best.

For our hyperparameter search we can use grid search or randomized search.

* **Grid search** tries every possible combination from lists of different parameters and determines which combination is most effective.<br><br>

* **Randomized search** will select random combinations of hyperparameters following defined rules or boundaries. This method is effective for very large search spaces, where trying out every combination is not computationally practical.

For this module, we will use grid search. Sklearn's [`GridSearchCV()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) class implements this method alongside cross-validation for a convenient hyperparameter search. See below for implementing GridSearchCV with the SGDRegressor. Randomized search is also available with [`RandomizedSearchCV()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). Finally, some models (e.g. ElasticNet) have more efficient hyperparameter search methods based on their mathematical characteristics (see [`ElasticNetCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html#sklearn.linear_model.ElasticNetCV)).

The output printed from this code simply describes the settings chosen for the grid search.

In [64]:
from sklearn.model_selection import GridSearchCV
# we create a dictionary of lists, each key is a parameter name
# and the list is the possible values it can take
# we have reduced the grid options to allow faster parameter searching
# for a larger search you can include 'l1' and 'elasticnet' penalties
# and 'constant' and 'optimal' learning_rates
param_grid = {
    'alpha': 10.0 ** -np.arange(4, 7),
    'loss': ['squared_error', 'huber', 'epsilon_insensitive'],
    'penalty': ['l2'],
    'learning_rate': ['invscaling'],
}
# we can bump up the max_iterations parameter, otherwise some of our models will fail to converge
clf = SGDRegressor(max_iter=10000, tol=1e-3)
# we pass the model, our parameter grid, and cross-validation parameters to the class
grid_search = GridSearchCV(estimator=clf, 
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',
                           cv=5)
# last, we fit our data. This will take a while...
grid_search.fit(X=X, y=np.ravel(y))

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",SGDRegressor(max_iter=10000)
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'alpha': array([1.e-04...e-05, 1.e-06]), 'learning_rate': ['invscaling'], 'loss': ['squared_error', 'huber', ...], 'penalty': ['l2']}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'neg_mean_squared_error'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation t

We can look at our `grid_search`'s `best_params_` and `best_estimator_` attributes to see which combination of hyperparameters was ultimately the most effective.

In [65]:
print(grid_search.best_params_)
print("\n",grid_search.best_estimator_)

{'alpha': np.float64(1e-05), 'learning_rate': 'invscaling', 'loss': 'squared_error', 'penalty': 'l2'}

 SGDRegressor(alpha=np.float64(1e-05), max_iter=10000)


<b style="color:red;">We can also print out the results of all of the models tested. Let's add a rule to filter down to only those which performed better than the default version we found originally.</b>

In [ ]:
cv_scores = grid_search.cv_results_
for mean_score, params in zip(cv_scores['mean_test_score'], cv_scores['params']):
    # how do we know 171.9 is the threshold to beat because our default mean score was 198.86602783825413
    if np.sqrt(-mean_score) < 171.9:
        print(np.sqrt(-mean_score), params)

212.7883284512471 {'alpha': np.float64(0.0001), 'learning_rate': 'invscaling', 'loss': 'squared_error', 'penalty': 'l2'}
227.13552644186507 {'alpha': np.float64(0.0001), 'learning_rate': 'invscaling', 'loss': 'huber', 'penalty': 'l2'}
221.23853217712133 {'alpha': np.float64(0.0001), 'learning_rate': 'invscaling', 'loss': 'epsilon_insensitive', 'penalty': 'l2'}
212.69130293393087 {'alpha': np.float64(1e-05), 'learning_rate': 'invscaling', 'loss': 'squared_error', 'penalty': 'l2'}
226.65091828945208 {'alpha': np.float64(1e-05), 'learning_rate': 'invscaling', 'loss': 'huber', 'penalty': 'l2'}
219.9445250790894 {'alpha': np.float64(1e-05), 'learning_rate': 'invscaling', 'loss': 'epsilon_insensitive', 'penalty': 'l2'}
212.74332310955413 {'alpha': np.float64(1e-06), 'learning_rate': 'invscaling', 'loss': 'squared_error', 'penalty': 'l2'}
226.59661744055418 {'alpha': np.float64(1e-06), 'learning_rate': 'invscaling', 'loss': 'huber', 'penalty': 'l2'}
219.75086182284994 {'alpha': np.float64(1e-

Now, let's grab our best model and do some performance testing. First, on our original test set, and then with our cross-validation methodology. Here the output will show the accuracy ($\text{RMSE}$) of the best model, as well as a sample of five predicted and actual scores.

In [79]:
final_model = grid_search.best_estimator_
display_results(final_model, X_test, y_test)

RMSE: 273.1140384244941
Predicted 1-5: [ 59.97298007  83.86134849 143.39072419  70.25582705  54.41622449]
Actual 1-5: [ 45.  38.  76.  69. 100.]


In [80]:
scores = cross_val_score(final_model, X, np.ravel(y), cv=5, scoring='neg_mean_squared_error')
display_scores(scores)

Scores: [117.50250221 316.27592567 238.94449438 203.66477646 118.23719757]
Mean: 198.92497925774
Standard deviation: 75.5452311836507


# Conclusion: Launch, Monitor, Maintain

Getting a model up an running is only the beginning of successful machine learning. For your model to be useful, it will likely need to be launched; set up for automated data ingestion and reporting; monitored for performance drops; and maintained or improved over time.

## Launch

First, always save any models you work on. Sklearn's [`joblib`](https://scikit-learn.org/stable/modules/model_persistence.html) provides methods to save models as a `.pkl` file (a standard python format for serializing data objects). Simply dump your model into a file, and load it back up when you need it or want to share it with a colleague.

In [83]:
import joblib
joblib.dump(final_model, "final_model.pkl")
final_model_loaded = joblib.load("final_model.pkl")

Since there is often new data available, writing code to automate the process of scraping new datasets and using them to further train your model will help it improve over time. Otherwise, performance can degrade over time as real word data and its underlying trends evolve.

## Monitor

If your model will be changing over time, it is essential to plan for regular testing. A wide range of scenarios can break your model, and you want to prepare for as many of them as possible. The introduction of new data features or removal of old features, changing data formats, and changes to the packages being used are just some examples of what could go wrong. Be sure to study up on version control and unit tests before deploying any machine learning model.

You will also want to monitor for performance. Having automated systems in place to calculate and report on performance metrics will allow for faster resolution of issues.

## Maintain

Ultimately, you will need some level of human intervention to not only keep your model running smoothly, but also to help it improve over time. When tests fail or performance drops, there will be many possible causes, so it is important to know who is responsible for checking in on any problems and fixing them as they arise. Ideally, you can automate the learning process. If you do, be sure to save versions of your model at regular intervals so you can compare between different versions and roll back to an older version if necessary. Upload your code to a public or private repository like GitHub to easily track changes that you or others make over time.

## Taking this further

If you're interested in exploring ways to improve the models used here, don't hesitate to play around! You can download additional data from the AirBnB website linked below. Try choosing a different city or a different time period.

You can also choose to include more features from the dataset. The `amenities` feature in particular contains a rich store of information, but in an inconvenient format. It would be excellent practice to take this feature, create a pre-processing pipeline that will turn it into usable numerical values, and include it in the models explored above. You will have to employ some Python string methods to parse the values contained in the feature, or you can use a text-based encoding method such as the `FeatureHasher()` or `CountVectorizer()`.

A more advanced model would consider how the AirBnB landscape changes over time. There might be trends picked up in a year's worth of data that are not seen within any one month. There may also be other interesting features that aren't found directly in the data. This incomplete [kaggle kernel](https://www.kaggle.com/rudymizrahi/how-i-ranked-5th-160-in-deloitte-s-ml-competition) suggests calculating the distance of each location from the downtown core.

**End of Module**

This notebook makes up one part of this module. Now that you have completed this part, please proceed to the next notebook in this module.

If you have any questions, please reach out to your peers using the discussion boards. If you and your peers are unable to come to a suitable conclusion, do not hesitate to reach out to your instructor on the designated discussion board.

# Exercise Solutions

## EXERCISE 4

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators=10)
rfr.fit(X_train, np.ravel(y_train))
print(rmse(rfr.predict(X), y))
print()
print(rfr.predict(X_test[0:5]))
print(y_test[0:5, 0])

In [ ]:
# Cross-validation
rfr = RandomForestRegressor(n_estimators=10)
scores = cross_val_score(rfr, X, np.ravel(y), cv=5, scoring='neg_mean_squared_error')
display_scores(scores)

# References

AirBnB. (2019). Detailed Listings data for Toronto, January 2019. Retrieved from http://insideairbnb.com/get-the-data.html.